In [1]:
ACCESS_KEY = 
SECRET_KEY = 
ENCODED_SECRET_KEY = SECRET_KEY.replace("/", "%2F")
AWS_BUCKET_NAME = "si618dimension"
MOUNT_NAME = "si618dimension"

# Comment the following line if you need to unmount the S3 bucket
# dbutils.fs.unmount("/mnt/si618dimension/")

# mount the S3 bucket
dbutils.fs.mount("s3a://%s:%s@%s" % (ACCESS_KEY, ENCODED_SECRET_KEY, AWS_BUCKET_NAME), "/mnt/%s" % MOUNT_NAME)

In [2]:
display(dbutils.fs.ls("/mnt/si618dimension/"))

In [3]:
heptathlon = spark.read.csv("/mnt/si618dimension/heptathlon.csv", inferSchema=True, header=True)

In [4]:
display(heptathlon)

In [5]:
import pandas as pd
from pyspark.ml.feature import VectorAssembler, StandardScaler, PCA
from pyspark.ml import Pipeline
from pyspark.ml.stat import Correlation
import matplotlib.pyplot as plt
import seaborn as sns
from pyspark.sql.functions import udf
from pyspark.sql.types import *
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.linalg import Vectors, VectorUDT
from pyspark.sql.functions import col

## 1. Perform Principal Component Analysis (PCA)

### 1.1 Assemble the Features

In [8]:
# list comprehension to include only the variables that are neither score nor athlete
feature_columns = [column for column in heptathlon.columns if column != 'Score' and column != 'Athlete']

# now concat into one vector
assembler = VectorAssembler(
    inputCols=feature_columns,
    outputCol="features")

### 1.2 Rescale the Data
Let us normalize the data by computing the "[z-scores](https://en.wikipedia.org/wiki/Standard_score#Calculation_from_raw_score)" of the values. This step of normalization (rescaling) is important to PCA.

Given a column x, the z-scores of the elements in x is simply

    (x - mean(x)) / std(x)
    
where, `std` means standard deviation.

In [10]:
# pre-process normalization
scaler = StandardScaler(inputCol="features", outputCol="scaledFeatures",
                        withStd=True, withMean=True)
#withMean=True suggests mean is subtracted

### 1.3 Perform Principal Component Analysis (PCA)

In [12]:
# k can't be higher than the number of variables. Max N of axis can be at most the number of original features.
pca = PCA(k=7, inputCol="scaledFeatures", outputCol="pcaFeatures")

### 1.4 Fit the Pipeline and Transform the Data

In [14]:
# hard to debug pipeline. Solution: run pipeline with the first function. If it works, add the second. Then the third.
pipeline = Pipeline(stages=[assembler, scaler, pca])
pipelineModel = pipeline.fit(heptathlon)
heptathlon_transformed = pipelineModel.transform(heptathlon)
# heptathlon_transformed has three new columns: assembled vector, scaled vectored, then scaled pca values.

### 1.5 Choose the optimal number of Principal Components (PC) to retain

In [16]:
pcaModel = pipelineModel.stages[-1]
pcaModel.explainedVariance
# this gives the percents that each pca value accounts for the explained variation of the factor

In [17]:
f, ax = plt.subplots(figsize=(5, 3))
plt.plot(range(1,8), pcaModel.explainedVariance, 'b-o')
display(f.figure)
# select three principle components for later analysis

## 2. Comparison of Original Features and Principal Components

### 2.1 Create a Correlation Matrix

#### Using Original Features

In [21]:
r1 = Correlation.corr(heptathlon_transformed, "features", "pearson").head()
print("Pearson correlation matrix:\n")
original_correlation = pd.DataFrame(r1[0].toArray(), columns=feature_columns)
original_correlation

#### Using Principal Components

In [23]:
r2 = Correlation.corr(heptathlon_transformed, "pcaFeatures", "pearson").head()
print("Pearson correlation matrix:\n")
PC_correlation = pd.DataFrame(r2[0].toArray(), columns=[str("PC"+str(i)) for i in range(1,8)])
PC_correlation

### 2.2 Draw a "clustered" heatmap
Refer to: https://seaborn.pydata.org/generated/seaborn.heatmap.html

#### Using Original Features

In [26]:
f, ax = plt.subplots()
sns.clustermap(original_correlation,figsize=(8,5))
display(f.figure)
#hierarchical clustering. Not using bisecting k means, but instead hierarchical clustering. Put two most correlated into one cluster, then add one more.

#### Using Principal Components

In [28]:
f, ax = plt.subplots()
sns.clustermap(PC_correlation,figsize=(8,5))
display(f.figure)
# The colors only show on diagonal -- principal components are orthogonal to eachother in geometric space. Linearly uncorrelated.

### 2.3 Predict Heptathlon Score and Evaluate Models using Cross Validation

#### Linear Regression Using Original Features

In [31]:
lr = LinearRegression(featuresCol="features",labelCol="Score")
paramGrid = ParamGridBuilder().build()
evaluator = RegressionEvaluator(labelCol="Score", predictionCol="prediction", metricName="rmse")

# can use cross val for two things: minimize errors, but then also tune the number of trees, depth, etc.
# here not interested in finding best parameters
crossval = CrossValidator(estimator=lr,
                          estimatorParamMaps=paramGrid,
                          evaluator=evaluator,
                          numFolds=5)
crossvalModel = crossval.fit(heptathlon_transformed)
crossvalModel.avgMetrics

#### Principal Component Regression Using All PCs

In [33]:
lr = LinearRegression(featuresCol="pcaFeatures",labelCol="Score")
paramGrid = ParamGridBuilder().build()
evaluator = RegressionEvaluator(labelCol="Score", predictionCol="prediction", metricName="rmse")

crossval = CrossValidator(estimator=lr,
                          estimatorParamMaps=paramGrid,
                          evaluator=evaluator,
                          numFolds=5)
crossvalModel = crossval.fit(heptathlon_transformed)
crossvalModel.avgMetrics
# similar result since we don't discard any information

#### Principal Component Regression Using 3 PCs

In [35]:
heptathlon_transformed.printSchema()

In [36]:
k = 3
select_first_k_PC = udf(lambda x: Vectors.dense([c for c in x[:k]]), VectorUDT())
heptathlon_transformed = heptathlon_transformed.withColumn("first_k_pcaFeatures", select_first_k_PC(col("features")))
display(heptathlon_transformed.select("first_k_pcaFeatures"))

In [37]:
lr = LinearRegression(featuresCol="first_k_pcaFeatures",labelCol="Score")
paramGrid = ParamGridBuilder().build()
evaluator = RegressionEvaluator(labelCol="Score", predictionCol="prediction", metricName="rmse")

crossval = CrossValidator(estimator=lr,
                          estimatorParamMaps=paramGrid,
                          evaluator=evaluator,
                          numFolds=5)
crossvalModel = crossval.fit(heptathlon_transformed)
crossvalModel.avgMetrics
# error is much much larger. When we do PCA, we do something on X, but not on Y. Possible that some variation with Y is discarded.

## End of Lab 11
## Nothing needs to be submitted